In [1]:
%load_ext django_orm_magic

In [2]:
%django_settings --default

Deleted custom settings. Back to default for %%django_orm


In [2]:
%%django_settings

DATABASES = {
  'default': {
    'ENGINE': 'django.db.backends.postgresql',
    'NAME': 'preciosclaros',
    'USER': 'preciosclaros',
    'PASSWORD': 'preciosclaros'  
  }
}
INSTALLED_APPS = ("orm_magic",)

Settings for %%django_orm configured succesfully


In [2]:
!head datasets/cadenas.csv

,comercioId,banderaId,comercioRazonSocial,banderaDescripcion
0,9,2,Jumbo Retail Argentina S.A.,Disco
1,9,3,Jumbo Retail Argentina S.A.,Jumbo
2,9,1,Jumbo Retail Argentina S.A.,Vea
14,23,1,Pan American Energy LLC Argentina,Axion Energy
33,11,5,Wal Mart Argentina S.R.L.,Walmart SuperCenter
35,11,1,Wal Mart Argentina S.R.L.,Walmart Supermercado
59,11,3,Wal Mart Argentina S.R.L.,Changomas Express
60,15,1,DIA Argentina S.A,Supermercados DIA
77,3,1,Deheza S.A.I.C.F. e I.,DEHEZA S.A.I.C.F. e I.


In [3]:
!head datasets/sucursales.csv

localidad,sucursalNombre,direccion,sucursalTipo,lat,lng,provincia,sucursalId,comercioId,banderaId
CIUDAD AUTONOMA BUENOS AIRES,SM 916 Gorostiaga,Gorostiaga Jose Benjamin. 1636,Supermercado,-34.5655,-58.4368,AR-C,916,9,2
CIUDAD AUTONOMA BUENOS AIRES,Jumbo Almagro,Guardia Vieja 4558,Hipermercado,-34.6,-58.4312,AR-C,5231,9,3
CIUDAD AUTONOMA BUENOS AIRES,Barracas,Herrera 803,Supermercado,-34.6382,-58.3788,AR-C,741,9,1
CIUDAD AUTONOMA BUENOS AIRES,Larrea.,Larrea 833,Supermercado,-34.5985,-58.4032,AR-C,10,9,1
CIUDAD AUTONOMA BUENOS AIRES,Mansilla.,Mansilla Gral Lucio Norberto 3640,Supermercado,-34.5909,-58.4172,AR-C,5,9,1
CIUDAD AUTONOMA BUENOS AIRES,SM Manzanares.,Manzanares Sold 3953,Supermercado,-34.5545,-58.4835,AR-C,665,9,1
CIUDAD AUTONOMA BUENOS AIRES,Montevideo.,Montivideo 1037,Supermercado,-34.5964,-58.3902,AR-C,137,9,2
CIUDAD AUTONOMA BUENOS AIRES,Naón.,Naon Dr. Romulo 2142,Supermercado,-34.5696,-58.473,AR-C,107,9,2
CIUDAD AUTONOMA BUENOS AIRES,Paraguay.,Paraguay 4302,Supermercado,-

In [4]:
!head datasets/productos.csv

In [5]:
!head datasets/precios_2018-04-20.csv

sucursal_id,precio,producto_id
9-2-12,27.39,7790080065107
9-2-12,22.45,7790080065183
9-2-12,22.99,7791337651227
9-2-12,40.0,7791337690820
9-2-12,27.39,7790080065121
9-2-12,19.99,7790080065152
9-2-12,19.99,7790080065169
9-2-12,22.45,7790080065190
9-2-12,22.99,7791337601215


In [3]:
%%django_orm
from django.db import models

class Empresa(models.Model): 
    # aka comercio
    external_id = models.PositiveIntegerField(null=True, blank=True)
    nombre = models.CharField(max_length=80)
    

class Cadena(models.Model):
    # aka Bandera
    external_id = models.PositiveIntegerField()
    empresa = models.ForeignKey('Empresa', on_delete=models.CASCADE)
    nombre = models.CharField(max_length=80)

    
class Sucursal(models.Model):
    external_id = models.CharField(max_length=20, primary_key=True)
    localidad = models.CharField(max_length=80)
    nombre = models.CharField(max_length=50)
    direccion = models.CharField(max_length=100)
    tipo = models.CharField(max_length=20)
    lat = models.FloatField()
    lng = models.FloatField()
    cadena = models.ForeignKey('Cadena', on_delete=models.CASCADE)
    
    
class Marca(models.Model):
    nombre = models.CharField(max_length=80, primary_key=True)
    empresa = models.ForeignKey('Empresa', null=True, on_delete=models.SET_NULL)
    

class Categoria(models.Model):
    nombre = models.CharField(max_length=80)
    
    
class Producto(models.Model):
    codigo = models.CharField(max_length=20, primary_key=True)
    nombre = models.CharField(max_length=150)
    marca = models.ForeignKey('Marca',  on_delete=models.CASCADE)
    categoria = models.ForeignKey('Categoria', null=True, on_delete=models.SET_NULL)
    presentacion = models.CharField(max_length=50)
    
class Relevamiento(models.Model):
    fecha = models.DateField()
    
    
class Precio(models.Model):
    relevamiento = models.ForeignKey(Relevamiento, on_delete=models.CASCADE)
    producto = models.ForeignKey(Producto, on_delete=models.CASCADE)
    sucursal = models.ForeignKey(Sucursal, on_delete=models.CASCADE)
    precio = models.FloatField()  # DecimalField(max_digits=6, decimal_places=2)

In [4]:
import pandas as pd

In [5]:
cadenas = pd.read_csv('datasets/cadenas.csv')

In [6]:
c = []
for row in cadenas.itertuples():
    empresa, _ = Empresa.objects.get_or_create(nombre=row.comercioRazonSocial, external_id=row.comercioId)
    c.append(Cadena(empresa=empresa, nombre=row.banderaDescripcion, external_id=row.banderaId))
Cadena.objects.bulk_create(c);

In [7]:
Sucursal.objects.all().delete()

(0, {})

In [8]:
sucs = pd.read_csv('datasets/sucursales.csv')
s = []
for row in sucs.drop_duplicates(['comercioId', 'banderaId', 'sucursalId']).itertuples():
    cadena = Cadena.objects.get(empresa__external_id=row.comercioId, external_id=row.banderaId)
    external_id = "{}-{}-{}".format(row.comercioId, row.banderaId, row.sucursalId)
    s.append(Sucursal(external_id=external_id, localidad=row.localidad, nombre=row.sucursalNombre, 
                      direccion=row.direccion, tipo=row.sucursalTipo, 
                      lat = row.lat, lng=row.lng, cadena=cadena))
Sucursal.objects.bulk_create(s);
                        

In [9]:
Producto.objects.all().delete()

(0, {})

In [10]:
prods = pd.read_csv('datasets/productos.csv')

In [11]:
Marca.objects.all().delete()
Marca.objects.bulk_create([Marca(nombre=m) for m in prods.marca.unique()]);

In [12]:
objects = []
for row in prods.drop_duplicates('id').itertuples():
    objects.append(Producto(codigo=row.id, nombre=row.nombre, presentacion=row.presentacion, marca_id=row.marca))
Producto.objects.bulk_create(objects);

In [13]:
Producto.objects.count()

55785

In [14]:
from itertools import zip_longest

def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)


In [15]:
from tqdm import tqdm_notebook

In [16]:
Relevamiento.objects.all().delete()

(0, {})

In [17]:
CHUNK_SIZE = int(20e3)

archivos = !ls datasets/precios_*.csv
fechas = [f[17:-4] for f in archivos ]

for fecha, csv in tqdm_notebook(zip(fechas, archivos), total=len(fechas)):
    relevamiento, _ = Relevamiento.objects.get_or_create(fecha=fecha)
    precios = pd.read_csv(csv)
    # sólo precios para productos conocidos
    precios = precios.loc[precios.producto_id.isin(prods.id)]
    
    expected_iterations = precios.size // CHUNK_SIZE + 1
    for group in tqdm_notebook(grouper(precios.itertuples(), CHUNK_SIZE), total=expected_iterations): 
        p = []
        for r in group:
            if not r:
                # terminamos! 
                break
            try:
                p.append(Precio(sucursal_id=r.sucursal_id, producto_id=r.producto_id, relevamiento=relevamiento, precio=r.precio))
            except Exception as e:
                print(e)
                continue
        Precio.objects.bulk_create(p)

/home/tin/.virtualenvs/curso/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


TypeError: '>' not supported between instances of 'str' and 'int'

In [24]:
precios.index.size

6071219

In [25]:
CHUNK_SIZE = int(30e3)

for fecha, csv in tqdm_notebook(zip(fechas[-1:], archivos[-1:]), total=1):
    relevamiento, _ = Relevamiento.objects.get_or_create(fecha=fecha)
    precios = pd.read_csv(csv)
    # sólo precios para productos conocidos
    precios = precios.loc[precios.producto_id.astype(str).isin(prods.id.astype(str))]
    
    expected_iterations = precios.index.size // CHUNK_SIZE + 1
    for group in tqdm_notebook(grouper(precios.itertuples(), CHUNK_SIZE), total=expected_iterations): 
        p = []
        for r in group:
            if not r:
                # terminamos! 
                break
            try:
                p.append(Precio(sucursal_id=r.sucursal_id, producto_id=r.producto_id, relevamiento=relevamiento, precio=r.precio))
            except Exception as e:
                print(e)
                continue
        Precio.objects.bulk_create(p)

/home/tin/.virtualenvs/curso/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
Precio.objects.count()

0

In [8]:
archivos = !ls datasets/precios_*.csv
fechas = [f[17:-4] for f in archivos ]

precios = pd.read_csv('datasets/precios_2018-04-20.csv')

In [12]:
relevamiento = Relevamiento.objects.get(fecha="2018-04-20")

In [10]:
csv, r

('datasets/precios_2018-04-20.csv',
 Pandas(Index=4607755, sucursal_id='17-1-256', precio=18.800000000000001, producto_id='7790220001705'))

In [14]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 592kB/s ta 0:00:01
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
precios[4607755:].size // 600

9200

In [29]:


for chunk in tqdm_notebook(grouper(precios[4607755:].itertuples(), 600), total=9200):
    p = []
    for r in tqdm_notebook(chunk, total=600, leave=False):
        try:
            producto = Producto.objects.get(codigo=r.producto_id)
        except Producto.DoesNotExist as e:
            print(e)
            continue
        try:
            sucursal = Sucursal.get_by_full_id(r.sucursal_id)
        except sucursal.DoesNotExist as e:
            print(e, r.sucursal_id)
            continue
        
        p.append(Precio(sucursal=sucursal, producto=producto, relevamiento=relevamiento, precio=r.precio))
        
    Precio.objects.bulk_create(p)

NameError: name 'sucursal' is not defined

In [17]:
relevamiento.precio_set.count(), Precio.objects.count()

(4751499, 4751499)